# RAG Basics with LlamaIndex & Qdrant
**Concept:** Index data into a Vector Database (Qdrant). Retrieve only the relevant chunks.

**Architecture:**
1. **Load:** Documents -> Nodes.
2. **Embed:** Nodes -> Vectors (via Ollama).
3. **Store:** Vectors -> Qdrant.
4. **Query:** Question -> Vector Search -> LLM.

In [ ]:
import qdrant_client
from llama_index.core import VectorStoreIndex, Document, Settings, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

LLM_MODEL="qwen3:0.6b"
EMB_MODEL="nomic-embed-text"
QDRANT_URL="http://localhost:6333"
COLLECTION_NAME="demo_rag_llamaindex"


# 1. Setup Global Settings (The Brains)
Settings.llm = Ollama(model=LLM_MODEL, request_timeout=60.0)
Settings.embed_model = OllamaEmbedding(model_name=EMB_MODEL)

# 2. Setup Vector Store (The Filing Cabinet)
# We connect to your docker Qdrant container
client = qdrant_client.QdrantClient(url=QDRANT_URL)

if client.collection_exists(COLLECTION_NAME):
  client.delete_collection(COLLECTION_NAME)
  print(f"🗑️ Deleted existing collection: {COLLECTION_NAME}")

# Create fresh connection
vector_store = QdrantVectorStore(client=client, collection_name=COLLECTION_NAME)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print("✨ Database is clean and ready.")

🗑️ Deleted existing collection: demo_rag_llamaindex
✨ Database is clean and ready.


In [2]:
# 3. Create Documents
documents = [
    Document(text="The European Union (EU) is a political and economic union of 27 member states.", metadata={"topic": "politics"}),
    Document(text="Python is a high-level, general-purpose programming language.", metadata={"topic": "tech"}),
    Document(text="Lisbon is the capital and largest city of Portugal.", metadata={"topic": "geography"}),
    Document(text="Generative AI refers to algorithms that can create new content.", metadata={"topic": "tech"}),
]

# 4. Indexing (The "Ingestion" Phase)
# This automatically chunks, embeds, and uploads to Qdrant
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

print("Documents indexed successfully into Qdrant!")

Documents indexed successfully into Qdrant!


In [3]:
# 5. The Query Engine (The Interface)
# We ask for the top 1 most similar chunk
query_engine = index.as_query_engine(similarity_top_k=1)

response = query_engine.query("What is the capital of Portugal?")
print(f"Answer: {response}\n")

# Check the source (Cite your work!)
print(f"Source Node: {response.source_nodes[0].node.text}")

Answer: Lisbon is the capital and largest city of Portugal.

Source Node: Lisbon is the capital and largest city of Portugal.
